In [1]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from ASC_ML import multiple_model_gen_v2 as multiple
from ASC_ML import dataframe_extractor as de
from ASC_ML import model_generation as model_gen
from ASC_ML import model_gen_train_test as testing
# from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-07-02 15:30:18.280696: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# m = multiple.Multiple_Model_Gen(max_no_layers = 2, model_per_batch = 10)

In [4]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [5]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [6]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [7]:
train_x = dataset_list[0].copy()

In [8]:
train_x = train_x.fillna(0)

In [9]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [10]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
# a = train_x['LotShape']

In [12]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [13]:
# train_x.head()

In [14]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [15]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [16]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [17]:
train_X = train_x.replace(keys).copy()

In [18]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [19]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [20]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [21]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [22]:
train_X = train_X.drop(["Id"], axis = 1)

In [23]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [24]:
len(train_X.columns)

58

In [25]:
len(dataset_list[1].columns)

1

In [26]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [27]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [28]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [29]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [30]:
import numpy as np
train_Y = np.asarray(dataset_list[1])

In [31]:
s = testing.Model_Parallel_Train_Test(train_X_scaled, train_Y_scaled, 58, 1, None)
# m = multiple.Multiple_Model_Gen_V2(train_X_scaled, train_Y_scaled, 50, 128, input_shape = 58, output_activation = "sigmoid", model_per_batch = 10)

In [32]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [33]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [34]:
# m.get_model_confs()
parallel_model = s.train_models()

2022-07-02 15:30:32.518494: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-02 15:30:32.520463: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-02 15:30:32.569470: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-02 15:30:32.569657: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-07-02 15:30:32.569679: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-02 15:30:32.571281: I tensorflow/stream_executor/platform/de

Epoch 1/100
12/12 [==============================] - 1s 4ms/step - loss: 0.3133 - output_layer_dense_4_4_4_loss: 0.0522 - output_layer_dense_8_8_8_loss: 0.1577 - output_layer_dense_16_16_16_loss: 0.0287 - output_layer_dense_32_32_32_loss: 0.0249 - output_layer_dense_64_64_64_loss: 0.0119 - output_layer_dense_128_128_128_loss: 0.0379 - output_layer_dense_4_4_4_mean_absolute_error: 0.1901 - output_layer_dense_8_8_8_mean_absolute_error: 0.3624 - output_layer_dense_16_16_16_mean_absolute_error: 0.1211 - output_layer_dense_32_32_32_mean_absolute_error: 0.1205 - output_layer_dense_64_64_64_mean_absolute_error: 0.0770 - output_layer_dense_128_128_128_mean_absolute_error: 0.1530


2022-07-02 15:30:34.255171: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 0.1279 - output_layer_dense_4_4_4_loss: 0.0192 - output_layer_dense_8_8_8_loss: 0.0761 - output_layer_dense_16_16_16_loss: 0.0114 - output_layer_dense_32_32_32_loss: 0.0084 - output_layer_dense_64_64_64_loss: 0.0051 - output_layer_dense_128_128_128_loss: 0.0077 - output_layer_dense_4_4_4_mean_absolute_error: 0.0970 - output_layer_dense_8_8_8_mean_absolute_error: 0.2485 - output_layer_dense_16_16_16_mean_absolute_error: 0.0751 - output_layer_dense_32_32_32_mean_absolute_error: 0.0662 - output_layer_dense_64_64_64_mean_absolute_error: 0.0511 - output_layer_dense_128_128_128_mean_absolute_error: 0.0626
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0838 - output_layer_dense_4_4_4_loss: 0.0133 - output_layer_dense_8_8_8_loss: 0.0492 - output_layer_dense_16_16_16_loss: 0.0069 - output_layer_dense_32_32_32_loss: 0.0059 - output_layer_dense_64_64_64_loss: 0.0038 - output_layer_dense_128_128_12

12/12 [==============================] - 0s 3ms/step - loss: 0.0245 - output_layer_dense_4_4_4_loss: 0.0072 - output_layer_dense_8_8_8_loss: 0.0100 - output_layer_dense_16_16_16_loss: 0.0027 - output_layer_dense_32_32_32_loss: 0.0021 - output_layer_dense_64_64_64_loss: 0.0011 - output_layer_dense_128_128_128_loss: 0.0013 - output_layer_dense_4_4_4_mean_absolute_error: 0.0609 - output_layer_dense_8_8_8_mean_absolute_error: 0.0701 - output_layer_dense_16_16_16_mean_absolute_error: 0.0361 - output_layer_dense_32_32_32_mean_absolute_error: 0.0312 - output_layer_dense_64_64_64_mean_absolute_error: 0.0233 - output_layer_dense_128_128_128_mean_absolute_error: 0.0247
Epoch 15/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0279 - output_layer_dense_4_4_4_loss: 0.0080 - output_layer_dense_8_8_8_loss: 0.0112 - output_layer_dense_16_16_16_loss: 0.0032 - output_layer_dense_32_32_32_loss: 0.0026 - output_layer_dense_64_64_64_loss: 0.0013 - output_layer_dense_128_128_128_loss: 0.0

...

12/12 [==============================] - 0s 3ms/step - loss: 0.0116 - output_layer_dense_4_4_4_loss: 0.0028 - output_layer_dense_8_8_8_loss: 0.0057 - output_layer_dense_16_16_16_loss: 0.0016 - output_layer_dense_32_32_32_loss: 9.9667e-04 - output_layer_dense_64_64_64_loss: 3.6665e-04 - output_layer_dense_128_128_128_loss: 2.1843e-04 - output_layer_dense_4_4_4_mean_absolute_error: 0.0341 - output_layer_dense_8_8_8_mean_absolute_error: 0.0517 - output_layer_dense_16_16_16_mean_absolute_error: 0.0264 - output_layer_dense_32_32_32_mean_absolute_error: 0.0208 - output_layer_dense_64_64_64_mean_absolute_error: 0.0138 - output_layer_dense_128_128_128_mean_absolute_error: 0.0101
Epoch 48/100
12/12 [==============================] - 0s 3ms/step - loss: 0.0142 - output_layer_dense_4_4_4_loss: 0.0033 - output_layer_dense_8_8_8_loss: 0.0069 - output_layer_dense_16_16_16_loss: 0.0020 - output_layer_dense_32_32_32_loss: 0.0013 - output_layer_dense_64_64_64_loss: 3.6288e-04 - output_layer_dense_128_1

12/12 [==============================] - 0s 4ms/step - loss: 0.0116 - output_layer_dense_4_4_4_loss: 0.0030 - output_layer_dense_8_8_8_loss: 0.0058 - output_layer_dense_16_16_16_loss: 0.0015 - output_layer_dense_32_32_32_loss: 9.5298e-04 - output_layer_dense_64_64_64_loss: 2.5152e-04 - output_layer_dense_128_128_128_loss: 1.8250e-04 - output_layer_dense_4_4_4_mean_absolute_error: 0.0343 - output_layer_dense_8_8_8_mean_absolute_error: 0.0496 - output_layer_dense_16_16_16_mean_absolute_error: 0.0254 - output_layer_dense_32_32_32_mean_absolute_error: 0.0199 - output_layer_dense_64_64_64_mean_absolute_error: 0.0115 - output_layer_dense_128_128_128_mean_absolute_error: 0.0101
Epoch 59/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0100 - output_layer_dense_4_4_4_loss: 0.0026 - output_layer_dense_8_8_8_loss: 0.0050 - output_layer_dense_16_16_16_loss: 0.0013 - output_layer_dense_32_32_32_loss: 7.4345e-04 - output_layer_dense_64_64_64_loss: 2.2848e-04 - output_layer_dense_1

12/12 [==============================] - 0s 3ms/step - loss: 0.0091 - output_layer_dense_4_4_4_loss: 0.0025 - output_layer_dense_8_8_8_loss: 0.0046 - output_layer_dense_16_16_16_loss: 0.0011 - output_layer_dense_32_32_32_loss: 6.9612e-04 - output_layer_dense_64_64_64_loss: 1.9497e-04 - output_layer_dense_128_128_128_loss: 7.7262e-05 - output_layer_dense_4_4_4_mean_absolute_error: 0.0321 - output_layer_dense_8_8_8_mean_absolute_error: 0.0437 - output_layer_dense_16_16_16_mean_absolute_error: 0.0227 - output_layer_dense_32_32_32_mean_absolute_error: 0.0176 - output_layer_dense_64_64_64_mean_absolute_error: 0.0100 - output_layer_dense_128_128_128_mean_absolute_error: 0.0063
Epoch 70/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0093 - output_layer_dense_4_4_4_loss: 0.0026 - output_layer_dense_8_8_8_loss: 0.0044 - output_layer_dense_16_16_16_loss: 0.0012 - output_layer_dense_32_32_32_loss: 7.4216e-04 - output_layer_dense_64_64_64_loss: 1.7995e-04 - output_layer_dense_1

12/12 [==============================] - 0s 4ms/step - loss: 0.0084 - output_layer_dense_4_4_4_loss: 0.0029 - output_layer_dense_8_8_8_loss: 0.0038 - output_layer_dense_16_16_16_loss: 9.4413e-04 - output_layer_dense_32_32_32_loss: 5.8629e-04 - output_layer_dense_64_64_64_loss: 1.4532e-04 - output_layer_dense_128_128_128_loss: 6.3932e-05 - output_layer_dense_4_4_4_mean_absolute_error: 0.0321 - output_layer_dense_8_8_8_mean_absolute_error: 0.0416 - output_layer_dense_16_16_16_mean_absolute_error: 0.0210 - output_layer_dense_32_32_32_mean_absolute_error: 0.0159 - output_layer_dense_64_64_64_mean_absolute_error: 0.0088 - output_layer_dense_128_128_128_mean_absolute_error: 0.0058
Epoch 81/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0080 - output_layer_dense_4_4_4_loss: 0.0025 - output_layer_dense_8_8_8_loss: 0.0037 - output_layer_dense_16_16_16_loss: 0.0010 - output_layer_dense_32_32_32_loss: 5.9490e-04 - output_layer_dense_64_64_64_loss: 1.4393e-04 - output_layer_den

12/12 [==============================] - 0s 4ms/step - loss: 0.0075 - output_layer_dense_4_4_4_loss: 0.0023 - output_layer_dense_8_8_8_loss: 0.0035 - output_layer_dense_16_16_16_loss: 0.0010 - output_layer_dense_32_32_32_loss: 4.9107e-04 - output_layer_dense_64_64_64_loss: 1.2170e-04 - output_layer_dense_128_128_128_loss: 3.3262e-05 - output_layer_dense_4_4_4_mean_absolute_error: 0.0323 - output_layer_dense_8_8_8_mean_absolute_error: 0.0389 - output_layer_dense_16_16_16_mean_absolute_error: 0.0223 - output_layer_dense_32_32_32_mean_absolute_error: 0.0153 - output_layer_dense_64_64_64_mean_absolute_error: 0.0077 - output_layer_dense_128_128_128_mean_absolute_error: 0.0041
Epoch 92/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0071 - output_layer_dense_4_4_4_loss: 0.0024 - output_layer_dense_8_8_8_loss: 0.0031 - output_layer_dense_16_16_16_loss: 9.2470e-04 - output_layer_dense_32_32_32_loss: 4.9257e-04 - output_layer_dense_64_64_64_loss: 1.0915e-04 - output_layer_den

In [38]:
parallel_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_dense_4_4_4 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_8_8_8 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_16_16 (Inp [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_32_32_32 (Inp [(None, 58)]         0                                            
______________________________________________________________________________________________

In [42]:
parallel_model.inputs

<KerasTensor: shape=(None, 58) dtype=float32 (created by layer 'input_layer_dense_4_4_4')>

In [43]:
parallel_model.outputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'output_layer_dense_4_4_4')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'output_layer_dense_8_8_8')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'output_layer_dense_16_16_16')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'output_layer_dense_32_32_32')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'output_layer_dense_64_64_64')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'output_layer_dense_128_128_128')>]

In [44]:
from tensorflow.keras.models import Model
model1 = Model(inputs = parallel_model.inputs[0], outputs = parallel_model.outputs[0])

In [45]:
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_4_4_4 (Inp [(None, 58)]              0         
_________________________________________________________________
layer1_dense_4_4_4 (Dense)   (None, 4)                 236       
_________________________________________________________________
layer2_dense_4_4_4 (Dense)   (None, 4)                 20        
_________________________________________________________________
layer3_dense_4_4_4 (Dense)   (None, 4)                 20        
_________________________________________________________________
output_layer_dense_4_4_4 (De (None, 1)                 5         
Total params: 281
Trainable params: 281
Non-trainable params: 0
_________________________________________________________________


In [47]:
model1.compile(loss = "mean_squared_error",
             optimizer = "adam",
             metrics = ["mean_absolute_error"])
model1.evaluate(train_X_scaled, train_Y_scaled)

46/46 [==============================] - 0s 770us/step - loss: 0.0018 - mean_absolute_error: 0.0286


[0.0022802152670919895, 0.029542429372668266]

In [56]:
sep_model_list = []
for i in range(6):
    sep_model_list.append(Model(inputs = parallel_model.inputs[i], outputs = parallel_model.outputs[i])) 


In [57]:
print(sep_model_list[5].weights)


[<tf.Variable 'layer1_dense_128_128_128/kernel:0' shape=(58, 128) dtype=float32, numpy=
array([[ 0.08779965,  0.17599675,  0.00166621, ...,  0.07710771,
         0.16700155, -0.02007297],
       [ 0.05864117,  0.16856018,  0.05949063, ...,  0.19368783,
        -0.14512852,  0.17768408],
       [-0.04151135, -0.01839427, -0.11705804, ...,  0.15131906,
         0.00610092,  0.05845013],
       ...,
       [ 0.01011156, -0.08880191, -0.05004855, ...,  0.1564212 ,
         0.00412588,  0.05678201],
       [ 0.05993765, -0.09355821,  0.03101114, ..., -0.08407053,
        -0.10902958, -0.04146596],
       [-0.0853507 , -0.1433334 ,  0.06186091, ...,  0.05932223,
         0.03734594, -0.15206495]], dtype=float32)>, <tf.Variable 'layer1_dense_128_128_128/bias:0' shape=(128,) dtype=float32, numpy=
array([ 4.33586910e-03, -8.80319532e-03, -1.85979884e-02, -4.42527654e-03,
        2.37734988e-03, -1.55048119e-02, -8.02167784e-03, -1.69753807e-03,
       -1.22372685e-02, -1.10868318e-03,  6.031707

In [58]:
sep_model_list[5].compile(loss = "mean_squared_error",
             optimizer = "adam",
             metrics = ["mean_absolute_error"])
sep_model_list[5].evaluate(train_X_scaled, train_Y_scaled)

46/46 [==============================] - 0s 889us/step - loss: 2.4459e-05 - mean_absolute_error: 0.0037


[2.6648578568710946e-05, 0.00365837081335485]

In [35]:
m.model_confs

NameError: name 'm' is not defined

In [ ]:
# m.get_all_models()
m.train_models()

In [ ]:
m.evaluate_dict_list

In [ ]:
# from ASC_ML import multiple_model_gen_v2 as multiple
# import sys
# sys.path.insert(0, '/home/anish/ASC_ML')

# m = multiple.Multiple_Model_Gen_V2(train_x, train_y, epochs, batch_size, input_shape, output_shape = 1, output_activation = None, model_per_batch = 10)
# m.train_models()
# m.evaluate_dict_list